In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, Random
using _Data, _Math

In [ ]:
@pyimport numpy
@pyimport sklearn.decomposition as decomposition
@pyimport sklearn.manifold as manifold
@pyimport matplotlib.cm as cm
@pyimport sklearn.metrics as metrics
@pyimport colorsys
#@pyimport netcomp as nc

include("../../project_place_cell/functions/func_map.jl")
include("../../project_place_cell/functions/func_stat.jl")
include("../../project_place_cell/functions/func_data.jl")
include("../../project_place_cell/functions/func_plot.jl")

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["axes.spines.top"] = false;
rc_params["axes.spines.right"] = false;

rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;

rc_params["axes.labelpad"] = 2;

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
#for quantification:

datasets = 
[
    ["20220406_111526", 9, "jen"],
    ["20220407_090156", 5, "jen"],
    ["20220406_153842", 9, "jen"],
    ["20220407_152537", 4, "jen"],
    ["20220417_165530", 2, "jen"],
    ["20220405_171444", 4, "jen"],
    ["20220416_160516", 6, "jen"]
];


chuyu_server = [9, 5, 9, 4, 2, 4, 6]

lengths = [90, 90, 90, 90, 89, 90, 90];
n_datasets = length(datasets)


In [ ]:
activity_bins = 7
activity_shift = 4
n_pos = 60 #number of bins in long side
long_axis_in_mm = 47 #47 for rectangular, 33 for others
use_amount = 1000
# file_name = "_decoding_revision.h5"


i=1

    experiment_filename = datasets[i][1]
    server = datasets[i][2]
    experimenter = datasets[i][end]
    analyzer = "lorenz"

    ds = Dataset(experiment_filename, experimenter, gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")
    ds_Chuyu = Dataset(experiment_filename, "chuyu", gethostname() == "roli-$(chuyu_server[i])" ? "/data" : "/nfs/data$(chuyu_server[i])")
    ds_Lorenz = Dataset(experiment_filename, "lorenz", gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")

In [ ]:

NMF_file = h5open(joinpath(data_path(ds_Chuyu), "NMF_merge.h5"))
    A_dFF = HDF5.readmmap(NMF_file["A_dFF"]);
    X_all = HDF5.readmmap(NMF_file["X_all"]);
    Y_all = HDF5.readmmap(NMF_file["Y_all"]);
    Z_all = HDF5.readmmap(NMF_file["Z_all"]);
close(NMF_file)

n_sweeps = size(A_dFF, 1)
n_neurons = size(A_dFF, 2)


img_bg= h5open(ds, "behavior.h5"; raw = true) do file
    read(file, "img_bg")
end;


info_file = h5open(ds_Chuyu, "neuron_spatial_info_0_90_chamber_geometry_$(experiment_filename)_sigma1_n60.h5", "r")
    specificity = HDF5.readmmap(info_file["specificity"])
    valid_neurons = HDF5.readmmap(info_file["valid_neurons"])
    place_cell_index = HDF5.readmmap(info_file["place_cell_index"])
    bool_index = HDF5.readmmap(info_file["bool_index"])
close(info_file)

A_dFF_place_cells = A_dFF[:,place_cell_index]

A_dFF_place_cells = py"ffill"(A_dFF_place_cells')
A_dFF_place_cells = py"bfill"(A_dFF_place_cells);

position_file = h5open(joinpath(data_path(ds_Chuyu), "for_place_calculation_chamber_geometry_$(experiment_filename)_n60.h5"))
    chamber_roi = read(position_file,"chamber_roi")
    x_fish_sweep_mean = read(position_file,"x_fish_sweep_mean")
    y_fish_sweep_mean = read(position_file,"y_fish_sweep_mean")
    speed_mm_s = read(position_file, "speed_mm_s")
    loc_digital = read(position_file, "loc_digital")
    x_digital = read(position_file, "x_digital")
    y_digital = read(position_file, "y_digital")
    x_bins = read(position_file, "x_bins")
    y_bins = read(position_file, "y_bins")
    valid_moving_indices = read(position_file, "valid_moving_indices")
close(position_file)

moving_valid = speed_mm_s .> 0.1;
x_fish_mm_fit = fit(TrendFilter, x_fish_sweep_mean .* 0.02, 5, 2);
y_fish_mm_fit = fit(TrendFilter, y_fish_sweep_mean .* 0.02, 5, 2);
speed_mm_s = (diff(x_fish_mm_fit.β).^2 .+ diff(y_fish_mm_fit.β).^2).^0.5 ./ 0.5;
speed_mm_s = vcat(speed_mm_s[1], speed_mm_s);


# load geometry
geometry_file = h5open(joinpath(data_path(ds_Chuyu), "chamber_geometry_$(experiment_filename).h5"))
    chamber_roi = read(geometry_file,"chamber_roi")
    countour_array = read(geometry_file,"countour_array")
    center_loc = read(geometry_file,"center_loc")
close(geometry_file)



fish_angle_center = angle_from(hcat(x_fish_sweep_mean,y_fish_sweep_mean), center_loc);
fish_angle_center[fish_angle_center.<0]=fish_angle_center[fish_angle_center.<0].+2*pi

fish_distance_center = distance_from(hcat(x_fish_sweep_mean,y_fish_sweep_mean), center_loc);
fish_distance_edge, closest_edge = distance_to_feature(hcat(x_fish_sweep_mean,y_fish_sweep_mean), countour_array);

## spot check data

In [ ]:
plot(bool_index)

In [ ]:
place_cell_index = intersect(place_cell_index, valid_neurons, findall(X_all.>700)) # will be replaced by real telecephalic neurons

plot_loc(place_cell_index)

In [ ]:
figure(figsize=(20,5))
subplot(1,4,1)
imshow(chamber_roi')
scatter(x_fish_sweep_mean,y_fish_sweep_mean, s=0.5)
scatter(center_loc[1], center_loc[2], color="r"); axis("off")

subplot(1,4,2)
imshow(chamber_roi')
scatter(x_fish_sweep_mean,y_fish_sweep_mean, s=0.5, c= fish_distance_center)
scatter(center_loc[1], center_loc[2], color="r"); axis("off")

subplot(1,4,3)
imshow(chamber_roi')
scatter(x_fish_sweep_mean,y_fish_sweep_mean, s= 1, c= fish_distance_edge)
scatter(center_loc[1], center_loc[2], color="r"); axis("off")

subplot(1,4,4)
imshow(chamber_roi')
scatter(x_fish_sweep_mean,y_fish_sweep_mean, s= 1, c= fish_angle_center)
scatter(center_loc[1], center_loc[2], color="r"); axis("off")

In [ ]:
# subsample occupancy

Random.seed!(3);
#set the masks all true for their area
start_mask = copy(valid_moving_indices)
start_mask[30*120:end] .= false
end_mask = copy(valid_moving_indices)
end_mask[1:60*120] .= false

# flip some trues in the mask to make it even
for l in unique(loc_digital)
    temp = intersect(findall(loc_digital .== l), findall(valid_moving_indices))
    start_occ = temp[temp .< 30*120]
    end_occ = temp[temp .> 60*120]

    d = length(start_occ) - length(end_occ)
    if d < 0
        end_mask[sample(end_occ, -d, replace=false)] .= false
    else
        start_mask[sample(start_occ, d, replace=false)] .= false
    end
end

plot(start_mask)
plot(end_mask)

## fit isomap example

In [ ]:
# define periods for manifold fitting
interval = 120*45
window_size = 120*45
windows_raw = [(((i-1)*interval+1):((i-1)*interval+window_size)) for i in 1:(floor(Int32, (n_sweeps-window_size)/interval)+1)];
if n_sweeps<10800
    append!(windows_raw, [120*45:n_sweeps])
end
windows_center = [mean(w) for w in windows_raw];
windows = [intersect(findall(bool_index), w) for w in windows_raw];

windows[1] = intersect(findall(start_mask), windows[1]);
windows[end] = intersect(findall(end_mask), windows[end]);




In [ ]:
# fit each period
isomap_all = []
isomap_embedding_all = []
@showprogress for (i, window) in enumerate(windows)
    isomap = manifold.Isomap(n_neighbors=100, n_components=2, p=1, n_jobs=-1)
    isomap.fit(A_dFF_place_cells'[window,:]);
    append!(isomap_all, [isomap])
end

# apply periods
@showprogress for i in 1:length(isomap_all)
    append!(isomap_embedding_all, [isomap_all[i].embedding_])
end


In [ ]:
function hsv_control(rgb_color, s, v)
    color_hsv = colorsys.rgb_to_hsv(rgb_color[1],rgb_color[2],rgb_color[3])
    color_rgb = colorsys.hsv_to_rgb(color_hsv[1],s,v)
    return color_rgb
end

hsv_cmap = cm.get_cmap("hsv", 360)

In [ ]:
rgb_color = hsv_cmap(Int32.(round.(180*fish_angle_center/pi)))

hsv_s = [1 for i in 1:size(rgb_color, 1)];
hsv_v = [1 for i in 1:size(rgb_color, 1)];
rgb_color_s = [hsv_control(rgb_color[i,:], hsv_s[i],hsv_v[i]) for i in 1:size(rgb_color, 1)];

In [ ]:
scatter(x_fish_sweep_mean,y_fish_sweep_mean, s=2, c=rgb_color_s)
imshow(img_bg[:,:,end]', cmap="gray")

In [ ]:
figure(figsize=(10,3))
for i in [1,2]
    subplot(1, 3, i)
    #start = windows_raw[i][1]
    window = windows[i]
    S_isomap = isomap_embedding_all[i]
    
    plot(S_isomap[:,1], S_isomap[:,2], color="gray", alpha=0.3, linewidth=0.3)
    
    rgb_color = hsv_cmap(Int32.(round.(180*fish_angle_center[window]/pi)))
    hsv_s = [(600 .-fish_distance_edge[window][i])/600 for i in 1:size(rgb_color, 1)];
    hsv_v = [(600 .-fish_distance_edge[window][i])/600+0.8 for i in 1:size(rgb_color, 1)];
    hsv_v[hsv_v.>1].=1
    rgb_color_s = [hsv_control(rgb_color[i,:], hsv_s[i],hsv_v[i]) for i in 1:size(rgb_color, 1)];

    scatter(S_isomap[:,1], S_isomap[:,2], s=1, c=rgb_color_s);

    axis("off")
end

tight_layout(pad=0.1)
savefig("F3a_isomap_plots_eq.pdf", format="pdf",  transparent=true, dpi=300)

In [ ]:
@pyimport pandas

df = pandas.DataFrame([isomap_embedding_all[1][:,1], isomap_embedding_all[1][:,2], isomap_embedding_all[2][:,1], isomap_embedding_all[2][:,2]])

df.to_csv("F3a_export_data.csv")

## one dataset neighborhood distance

In [ ]:
# local neighbor distances in physical space
#nearest_k = 30
l1 = []
l2 = []

k_range = [5, 10, 20, 30, 50, 100, 250, 500, 1000, 2000, 3000]

for nearest_k in k_range

    l1_distance_mean_all = []
    l2_distance_mean_all = []
    @showprogress for (i, window) in enumerate(windows)
        start = windows_raw[i][1]
        S_isomap = isomap_embedding_all[i]

        pair_distance_position = metrics.pairwise_distances(hcat(x_fish_sweep_mean,y_fish_sweep_mean)[window,:]);
        pair_distance_isomap = metrics.pairwise_distances(S_isomap);
        l1_distance_all = []
        l2_distance_all = []
        for which_frame in 1:size(pair_distance_position,1)
            nearest_k_index = sortperm(pair_distance_isomap[:,which_frame])[2:nearest_k+1]
            nearest_k_distance = pair_distance_position[nearest_k_index, which_frame]
            l1_distance = numpy.mean(nearest_k_distance)
            l2_distance = numpy.sqrt(numpy.mean(numpy.square(nearest_k_distance)))
            append!(l1_distance_all, l1_distance)
            append!(l2_distance_all, l2_distance)
        end
        l1_distance_mean = median(l1_distance_all)
        l2_distance_mean = median(l2_distance_all)
        append!(l1_distance_mean_all, l1_distance_mean)
        append!(l2_distance_mean_all, l2_distance_mean)
    end
        append!(l1, [l1_distance_mean_all])
        append!(l2, [l2_distance_mean_all])

end

In [ ]:
# compute baseline by random selection
nearest_k = 100
l1_distance_mean_all_baseline = []
l2_distance_mean_all_baseline = []
@showprogress for window in windows
    pair_distance_position = metrics.pairwise_distances(hcat(x_fish_sweep_mean,y_fish_sweep_mean)[window, :]);
    pair_distance_isomap = metrics.pairwise_distances(A_dFF_place_cells'[window, :])
    l1_distance_all = []
    l2_distance_all = []
    for which_frame in 1:size(pair_distance_position, 1)
        nearest_k_index = numpy.random.choice(1:size(pair_distance_position,1), nearest_k, replace=false)
        nearest_k_distance = pair_distance_position[nearest_k_index, which_frame]
        l1_distance = numpy.mean(nearest_k_distance)
        l2_distance = numpy.sqrt(numpy.mean(numpy.square(nearest_k_distance)))
        append!(l1_distance_all, l1_distance)
        append!(l2_distance_all, l2_distance)
    end
    l1_distance_mean = median(l1_distance_all)
    l2_distance_mean = median(l2_distance_all)
    append!(l1_distance_mean_all_baseline, l1_distance_mean)
    append!(l2_distance_mean_all_baseline, l2_distance_mean)
end

In [ ]:
l1_distance_mean_all_baseline

In [ ]:
plot(k_range, getindex.(l1, 1)*20*10^-3)
plot(k_range, getindex.(l1, 2)*20*10^-3)
axhline(l1_distance_mean_all_baseline[1]*20*10^-3)
axhline(l1_distance_mean_all_baseline[2]*20*10^-3, color="orange")

ylabel("neighborhood distance (mm)")
xlabel("neighbors")

In [ ]:
figure(figsize=(5,3))
plot(k_range, (getindex.(l1, 2) .- getindex.(l1, 1)) *20*10^-3)
axhline((l1_distance_mean_all_baseline[2] - l1_distance_mean_all_baseline[1]) *20*10^-3, c="black")
ylabel("late - early");#ylim(-1, 0.2)
xlabel("neighbors")

In [ ]:
fig, ax = subplots(1,1, figsize=(2,3))
ax.plot(windows_center[[1,2]], (l2_distance_mean_all[[1,2]])*20*10^-3)
ax.plot(windows_center[[1,2]], l2_distance_mean_all_baseline[[1,2]]*20*10^-3, color="grey")
ax.set_ylabel("Neighbors distance (mm)")

ax.spines["top"].set_visible(false)
ax.spines["right"].set_visible(false)

xticks([15*120, 45*120,75*120], labels=["0-30min","30-60min","60-90min"], rotation=45)

ax.set_ylim(0, 25)
ax.set_xlim(0,10800)

## multiple datasets neighbor distance

In [ ]:
l1_all = []
l2_all = []
mean_all_base = []
std_all_base = []
ranges = []



for i in 1:length(datasets)
    
    experiment_filename = datasets[i][1]
    server = datasets[i][2]
    experimenter = datasets[i][end]
    analyzer = "lorenz"

    ds = Dataset(experiment_filename, experimenter, gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")
    ds_Chuyu = Dataset(experiment_filename, "chuyu", gethostname() == "roli-$(chuyu_server[i])" ? "/data" : "/nfs/data$(chuyu_server[i])")
    ds_Lorenz = Dataset(experiment_filename, "lorenz", gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")
    
    
    println(readdir(data_path(ds)))
    
    NMF_file = h5open(joinpath(data_path(ds_Chuyu), "NMF_merge.h5"))
        A_dFF = HDF5.readmmap(NMF_file["A_dF"]);
        X_all = HDF5.readmmap(NMF_file["X_all"]);
        Y_all = HDF5.readmmap(NMF_file["Y_all"]);
        Z_all = HDF5.readmmap(NMF_file["Z_all"]);
        neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
    close(NMF_file)

    n_sweeps = size(A_dFF, 1)
    n_neurons = size(A_dFF, 2)


    img_bg= h5open(ds, "behavior.h5"; raw = true) do file
        read(file, "img_bg")
    end;


    info_file = h5open(ds_Chuyu, "neuron_spatial_info_15_$(lengths[i])_chamber_geometry_$(experiment_filename)_sigma1_n60_A_dF.h5", "r")
        specificity = HDF5.readmmap(info_file["specificity"])
        valid_neurons = HDF5.readmmap(info_file["valid_neurons"])
        place_cell_index = HDF5.readmmap(info_file["place_cell_index"])
        bool_index = HDF5.readmmap(info_file["bool_index"])
    
    close(info_file)

    
    file = h5open(joinpath(data_path(ds_Chuyu), "region_roi_bool.h5"))
        region_names = read(file, "region_names")
        region_roi_bool = read(file, "region_roi_bool")
    close(file)


    tel_index = findall(region_names .== "Telencephalon -")[1];
    mask_tel = falses(n_neurons)

    for which_neuron in 1:n_neurons
        mask_tel[which_neuron] = maximum(region_roi_bool[neuron_label.==which_neuron, tel_index])
    end

    mask_tel = findall(mask_tel)

    place_cell_index = intersect(place_cell_index, mask_tel)
    
    
    A_dFF_place_cells = A_dFF[:,place_cell_index]

    A_dFF_place_cells = py"ffill"(A_dFF_place_cells')
    A_dFF_place_cells = py"bfill"(A_dFF_place_cells);

    position_file = h5open(joinpath(data_path(ds_Chuyu), "for_place_calculation_chamber_geometry_$(experiment_filename)_n60.h5"))
        chamber_roi = read(position_file,"chamber_roi")
        x_fish_sweep_mean = read(position_file,"x_fish_sweep_mean")
        y_fish_sweep_mean = read(position_file,"y_fish_sweep_mean")
        speed_mm_s = read(position_file, "speed_mm_s")
        loc_digital = read(position_file, "loc_digital")
        x_digital = read(position_file, "x_digital")
        y_digital = read(position_file, "y_digital")
        x_bins = read(position_file, "x_bins")
        y_bins = read(position_file, "y_bins")
        valid_moving_indices = read(position_file, "valid_moving_indices")
    close(position_file)

    moving_valid = speed_mm_s .> 0.1;
    x_fish_mm_fit = fit(TrendFilter, x_fish_sweep_mean .* 0.02, 5, 2);
    y_fish_mm_fit = fit(TrendFilter, y_fish_sweep_mean .* 0.02, 5, 2);
    speed_mm_s = (diff(x_fish_mm_fit.β).^2 .+ diff(y_fish_mm_fit.β).^2).^0.5 ./ 0.5;
    speed_mm_s = vcat(speed_mm_s[1], speed_mm_s);


    # load geometry
    geometry_file = h5open(joinpath(data_path(ds_Chuyu), "chamber_geometry_$(experiment_filename).h5"))
        chamber_roi = read(geometry_file,"chamber_roi")
        countour_array = read(geometry_file,"countour_array")
        center_loc = read(geometry_file,"center_loc")
    close(geometry_file)



    fish_angle_center = angle_from(hcat(x_fish_sweep_mean,y_fish_sweep_mean), center_loc);
    fish_angle_center[fish_angle_center.<0]=fish_angle_center[fish_angle_center.<0].+2*pi

    fish_distance_center = distance_from(hcat(x_fish_sweep_mean,y_fish_sweep_mean), center_loc);
    fish_distance_edge, closest_edge = distance_to_feature(hcat(x_fish_sweep_mean,y_fish_sweep_mean), countour_array);

    
    
    

    # subsample occupancy

    Random.seed!(3);
    #set the masks all true for their area
    start_mask = copy(valid_moving_indices)
    start_mask[30*120:end] .= false
    end_mask = copy(valid_moving_indices)
    end_mask[1:60*120] .= false

    # flip some trues in the mask to make it even
    for l in unique(loc_digital)
        temp = intersect(findall(loc_digital .== l), findall(valid_moving_indices))
        start_occ = temp[temp .< 30*120]
        end_occ = temp[temp .> 60*120]

        d = length(start_occ) - length(end_occ)
        if d < 0
            end_mask[sample(end_occ, -d, replace=false)] .= false
        else
            start_mask[sample(start_occ, d, replace=false)] .= false
        end
    end


    
    windows = [findall(start_mask), findall(end_mask)]


    # fit each period
    isomap_all = []
    isomap_embedding_all = []
    @showprogress for (i, window) in enumerate(windows)
        isomap = manifold.Isomap(n_neighbors=100, n_components=2, p=1, n_jobs=-1)
        isomap.fit(A_dFF_place_cells'[window,:]);
        append!(isomap_all, [isomap])
    end

    # apply periods
    @showprogress for i in 1:length(isomap_all)
        append!(isomap_embedding_all, [isomap_all[i].embedding_])

        #append!(isomap_embedding_all, [isomap_all[2].transform(A_dFF_place_cells'[windows[i],:])])
    end

    # local neighbor distances in physical space
    # nearest_k = 30
    l1 = []

    k_range = [30]

    for nearest_k in k_range

        l1_distance_mean_all = []
        l2_distance_mean_all = []
        for (i, window) in enumerate(windows)
            #window = windows[1]##############################
            S_isomap = isomap_embedding_all[i]

            pair_distance_position = metrics.pairwise_distances(hcat(x_fish_sweep_mean,y_fish_sweep_mean)[window,:]);
            pair_distance_isomap = metrics.pairwise_distances(S_isomap); #metrics.pairwise_distances(A_dFF_place_cells'[window, :])# 
            l1_distance_all = []
            l2_distance_all = []
            for which_frame in 1:size(pair_distance_position,1)
                nearest_k_index = sortperm(pair_distance_isomap[:,which_frame])[2:nearest_k+1]
                nearest_k_distance = pair_distance_position[nearest_k_index, which_frame]
                l1_distance = numpy.mean(nearest_k_distance)
                
                append!(l1_distance_all, l1_distance)
                
            end
            l1_distance_mean = l1_distance_all# median(l1_distance_all)
            
            append!(l1_distance_mean_all, [l1_distance_mean])
            
        end
            append!(l1, [l1_distance_mean_all])

    end
    
    append!(l1_all, [l1])
    append!(ranges, [k_range])
    
    
    
    
    # compute baseline by random selection
    nearest_k = 100
    distance_mean_all_baseline = []
    distance_std_all_baseline = []
    
    for (i, window) in enumerate(windows)
        #window = windows[1]##############################
        S_isomap = isomap_embedding_all[i]
        pair_distance_position = metrics.pairwise_distances(hcat(x_fish_sweep_mean,y_fish_sweep_mean)[window, :]);
        pair_distance_isomap = metrics.pairwise_distances(A_dFF_place_cells'[window, :])# metrics.pairwise_distances(S_isomap);#metrics.pairwise_distances(A_dFF_place_cells'[window, :])
        distance_mean_all = []        
        
        for which_frame in 1:size(pair_distance_position, 1)
            nearest_k_index = numpy.random.choice(1:size(pair_distance_position,1), nearest_k, replace=false)
            nearest_k_distance = pair_distance_position[nearest_k_index, which_frame]
            distance_mean = numpy.mean(nearest_k_distance)
            distance_std = numpy.std(nearest_k_distance)
            
            append!(distance_mean_all, distance_mean)
            
        end
        
        append!(distance_mean_all_baseline, mean(distance_mean_all))
        append!(distance_std_all_baseline, std(distance_mean_all))
        
    end
    
    append!(mean_all_base, [distance_mean_all_baseline])
    append!(std_all_base, [distance_std_all_baseline])
    
    
end

In [ ]:
mean_all_base
std_all_base

In [ ]:
@pyimport scipy
a = []
for i in 1:n_datasets
    push!(a, scipy.stats.mannwhitneyu(l1_all[i][1][1] .- mean_all_base[i][1] , l1_all[i][1][2] .- mean_all_base[i][2] , alternative="greater"))
end

In [ ]:
a

In [ ]:
start = [mean(l1_all[i][1][1]) for i in 1:length(l1_all)]
ende = [mean(l1_all[i][1][2]) for i in 1:length(l1_all)]

start_base = [mean_all_base[i][1] for i in 1:length(l1_all)]
ende_base = [mean_all_base[i][2] for i in 1:length(l1_all)];

In [ ]:
start.*20*10^-3

In [ ]:
ende.*20*10^-3

In [ ]:
start_base.*20*10^-3

In [ ]:
ende_base.*20*10^-3

In [ ]:
fig, ax = subplots(figsize=(0.8,1.5))

ax.scatter(fill(1, length(start)), start.*20*10^-3, color="blue", s=2)
ax.scatter(fill(2, length(start)), ende.*20*10^-3, color="blue", s=2)

for i in 1:length(start)
    ax.plot([1,2], [start[i], ende[i]].*20*10^-3, linewidth=0.5, color="grey", zorder=-1)
end

ax.scatter(fill(1, length(start)), start_base.*20*10^-3, color="black", s=2)
ax.scatter(fill(2, length(start)), ende_base.*20*10^-3, color="black", s=2)

for i in 1:length(start)
    ax.plot([1,2], [start_base[i], ende_base[i]].*20*10^-3, linewidth=0.5, color="grey", zorder=-1)
end


ax.set_ylim(0, 30)
ax.set_xlim(0.8, 2.2)
ax.set_yticks([0, 10, 20, 30], labels=["0", "", "", "30"])
ax.set_xticks([1,2], labels=["Early", "Late"]) # labels=["Activity", "0.5 * Activity"], rotation=45, rotation_mode="anchor", ha="right")
ax.set_ylabel("Physical distance (mm)", labelpad=-5)
tight_layout(pad=0.1)
savefig("F3b_neighbor_distance.pdf", format="pdf",  transparent=true, dpi=300)